In [66]:
import pandas as pd
#导入线性回归
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import numpy as np

In [2]:
df1 = pd.read_excel('../因子数据整合/正交因子.xlsx')

### 核心原因在于选股因子间的截面相关性较不稳定。

### 选股因子截面相关性在本文中具体指在某一时点上，某两个选股因子截面因子值之间的线性相关性。

### 逐步添加变量，逐步正交

1） 对于正交顺序第一的因子，其正交因子值就等于其原始因子值；

2） 对于正交顺序第 k（k>1）的因子，将其原始因子值作为回归的因变量，现有的
已经正交过的因子作为回归的自变量，取回归残差为正交因子 k 的因子值；

3） 将正交因子 k 放入已经正交过的因子集合并对于正交顺序为 k+1 的因子进行步
骤（2）、（3）的处理。

In [53]:
df2 = df1.dropna()

In [110]:
def get_zj_matrix(df2):
    varlist = df2.columns.tolist()[3:]
    df3 =pd.DataFrame()
    df3[varlist[0]] = df2[varlist[0]]
    for i in range(1,len(varlist)):
        y = df2[varlist[i]].values.reshape(-1,1)
        x = df3.values
#         print(x.shape)
#         print(y.shape)
        model = LinearRegression()
        #train
        model.fit(x,y)
        beta = model.coef_
        alpha = model.intercept_
        print(alpha)
        resid = (y - np.dot(x,beta.T)-alpha)
        df3[varlist[i]] = resid
    return df3

In [112]:
df3 = get_zj_matrix(df2)

[5.6737521]
[54.32040992]
[144.10262664]
[62.1915065]


In [116]:
df3['Date'] = df2['Date'] 
df3['bond'] = df2['bond'] 

In [118]:
df3['integrated_var'] = (1/3)*df3['M1']+(1/3)*df3['return_on_valuation']+(1/9)*df3['debt2asset']+(1/9)*df3['roe_growthrate']+(1/9)*df3['netprofit_growthrate']

In [121]:
df3[['Date','bond','integrated_var']].to_excel('正交因子.xlsx')